Auto-Clean-Test

In [1]:
import logging
import os
import shutil
import dill as pickle
import pandas as pd
import matplotlib.pyplot as plt
from streamline.runners.auto_runner import AutoRunner
from streamline.runners.clean_optimize_runner import OptimizeClean

/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open("./data/DigenData/all_new_test_sets.pkl",'rb') as f:
    digennoise = pickle.load(f)

#print(digennoise.keys())

with open("./data/DigenData/all_new_test_sets_no_noise.pkl",'rb') as f:
    digen = pickle.load(f)

#print(digen.keys())
#print(digen['digen8_4426'])
Test= pd.DataFrame(digennoise['digen8_4426']['X'])
yval = pd.DataFrame(digennoise['digen8_4426']['y'])
current_noise = digennoise['digen8_4426']['noise']
yval.columns = ['y']
yval.rename(columns = {'0':'y'}, inplace = True)
out = pd.concat([Test, yval], axis=1)
out.index.name = 'InstanceId'
out.to_csv('./data/DemoData/digen8_4426.csv')
print(out)

                  X0        X1        X2        X3        X4        X5  \
InstanceId                                                               
0           1.764052  0.400157  0.978738  2.240893  1.867558 -0.977278   
1           0.144044  1.454274  0.761038  0.121675  0.443863  0.333674   
2          -2.552990  0.653619  0.864436 -0.742165  2.269755 -1.454366   
3           0.154947  0.378163 -0.887786 -1.980796 -0.347912  0.156349   
4          -1.048553 -1.420018 -1.706270  1.950775 -0.509652 -0.438074   
...              ...       ...       ...       ...       ...       ...   
9995       -0.030453  1.082681 -0.530580  0.602367 -2.138291 -0.056632   
9996        2.520830 -0.124884  0.018068 -0.998623 -1.262734 -0.565107   
9997       -0.545705  1.696766 -0.508186  0.003621 -1.018515  0.251562   
9998       -0.367787 -0.568358 -0.721248  0.198280 -0.217422  0.914757   
9999        0.428571 -0.164999  0.864963 -0.647662  0.720058 -0.337715   

                  X6        X7       

In [3]:
cleanrun = OptimizeClean(dataset_name=['digen8_4426'], n_trials=100)
cleanrun.run()
print(cleanrun.param)


[I 2023-06-30 15:37:11,862] A new study created in memory with name: no-name-aaece1c6-6097-4e07-8cd8-51023c1d1061
  0%|          | 0/100 [00:00<?, ?it/s]INFO: ------------------------------------------------------- 
INFO: Loading Dataset: digen8_4426
INFO: Validating and Identifying Feature Types...
INFO: Running Initial EDA:
INFO: Initial Data Counts: ----------------
INFO: Instance Count = 10000
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       5173
1      0       4827
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature

metadata.pickle digen8_4426
logs digen8_4426
digen8_4426 digen8_4426


INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
Best trial: 0. Best value: 0.98634:   2%|▏         | 2/100 [02:14<1:30:45, 55.57s/it] INFO: ------------------------------------------------------- 
INFO: Loading Dataset: digen8_4426
INFO: Validating and Identifying Feature Types...
INFO: Running Initial EDA:
INFO: Initial Data Counts: ----------------
INFO: Instance Count = 10000
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       5173
1      0       4827


EXCEPTION


INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
5               X0                 X5    -0.023813
15              X1                 X5    -0.022718
23              X2                 X3     0.022538
27              X2                 X7     0.019607
68              X6                 X8    -0.018813
18              X1                 X8    -0.018646
56              X5                 X6    -0.018500
28              X2                 X8    -0.016195
59              X5                 X9    -0.015959
8               X0                 X8    -0.012363
INFO: No Features with correlation higher that parameter
INFO: Running Basic Exploratory Analysis...
INFO: Processed Data Counts

...

INFO: Initial Data Counts: ----------------
INFO: Instance Count = 10000
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       5173
1      0       4827
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
5               X0                 X5    -0.023813
15              X1                 X5    -0.022718
23              X2                 X3     0.022538
27              X2                 X7     0.019607
68              X6                 X8    -0.018813
18              X1                 X8   

metadata.pickle digen8_4426
logs digen8_4426
digen8_4426 digen8_4426


INFO: Initial Data Counts: ----------------
INFO: Instance Count = 10000
INFO: Feature Count = 10
INFO:     Categorical  = 0
INFO:     Quantitative = 10
INFO: Missing Count = 0
INFO:     Missing Percent = 0.0
INFO: Class Counts: ----------------
INFO: Class Count Information
INFO: 
   Class  Instances
0      1       5173
1      0       4827
INFO: No textual categorical features, skipping label encoding
INFO: Running Feature Engineering
INFO: No Features with high missingness found
INFO: Not removing any features due to high missingness
INFO: No non-binary categorical features, skipping categorical encoding
INFO: Top 10 Correlated Features
INFO: 
   Removed_Feature Correlated_Feature  Correlation
5               X0                 X5    -0.023813
15              X1                 X5    -0.022718
23              X2                 X3     0.022538
27              X2                 X7     0.019607
68              X6                 X8    -0.018813
18              X1                 X8   

metadata.pickle digen8_4426
logs digen8_4426
digen8_4426 digen8_4426
{'categorical_cutoff': 5, 'sig_cutoff': 0.019871563009945022, 'featureeng_missingness': 0.7000000000000001, 'cleaning_missingness': 0.6000000000000001, 'correlation_removal_threshold': 0.9, 'exploration_list': ['Describe', 'Univariate Analysis', 'Feature Correlation'], 'partition_method': 'Random', 'n_splits': 9}


<Figure size 600x600 with 0 Axes>

categorical cut = 5
sig cutoff = 0.0198
feature missing = 0.7
cleaning miss = 0.6
correlation removal = 0.9
partition method: random
n_splits = 9

424 min runtime for 1 dataset with 100 trials. 
= 7 hours.

In [4]:
# autotest = AutoRunner(dataset_names=['digen8_4426'], clean=False, class_label='y', instance_label='InstanceId')

In [5]:
#output_csv = autotest.run()

In [6]:
#print(output_csv)

#performance = pd.read_csv(output_csv)
#print(performance)
#print(performance['ROC AUC'].max())
#print(performance.loc[performance['ROC AUC'].idxmax()][0])